In [ ]:
from pandas.plotting import scatter_matrix
from numpy.core.umath_tests import inner1d
from six.moves import urllib
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.base import TransformerMixin #gives fit_transform method for free
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
from sklearn import svm


import hashlib
import numpy as np
import os
import pandas as pd
import tarfile
import matplotlib.pyplot as plt

#Paths
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = "datasets/housing"
HOUSING_URL = DOWNLOAD_ROOT + HOUSING_PATH + "/housing.tgz"
#


##Helper functions
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
 if not os.path.isdir(housing_path):
  os.makedirs(housing_path)
 tgz_path = os.path.join(housing_path, "housing.tgz")
 urllib.request.urlretrieve(housing_url, tgz_path)
 housing_tgz = tarfile.open(tgz_path)
 housing_tgz.extractall(path=housing_path)
 housing_tgz.close()
def load_housing_data(housing_path=HOUSING_PATH):
 csv_path = os.path.join(housing_path, "housing.csv")
 return pd.read_csv(csv_path)
def split_train_test(data, test_ratio):
 shuffled_indices = np.random.permutation(len(data))
 test_set_size = int(len(data) * test_ratio)
 test_indices = shuffled_indices[:test_set_size]
 train_indices = shuffled_indices[test_set_size:]
 return data.iloc[train_indices], data.iloc[test_indices]
def test_set_check(identifier, test_ratio, hash):
 return hash(np.int64(identifier)).digest()[-1] < 256 * test_ratio
def split_train_test_by_id(data, test_ratio, id_column, hash=hashlib.md5):
 ids = data[id_column]
 in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio, hash))
 return data.loc[~in_test_set], data.loc[in_test_set]
def display_scores(scores):
 print("Scores:", scores)
 print("Mean:", scores.mean())
 print("Standard deviation:", scores.std())
##







##Combined Attributes Adder
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
 def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
  self.add_bedrooms_per_room = add_bedrooms_per_room
 def fit(self, X, y=None):
  return self # nothing else to do
 def transform(self, X, y=None):
  rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
  population_per_household = X[:, population_ix] / X[:, household_ix]
  if self.add_bedrooms_per_room:
   bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
   return np.c_[X, rooms_per_household, population_per_household,
       bedrooms_per_room]
  else:
   return np.c_[X, rooms_per_household, population_per_household]
#
#Label Binarizer Class    
class MyLabelBinarizer(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.encoder = LabelBinarizer(*args, **kwargs)
    def fit(self, x, y=0):
        self.encoder.fit(x)
        return self
    def transform(self, x, y=0):
        return self.encoder.transform(x) 
#       
#The DataFrameSelector Class
class DataFrameSelector(BaseEstimator, TransformerMixin):
 def __init__(self, attribute_names):
  self.attribute_names = attribute_names
 def fit(self, X, y=None):
  return self
 def transform(self, X):
  return X[self.attribute_names].values
##


#Plotting frequencey histograms of each data attribute
housing = load_housing_data()
housing.hist(bins=50, figsize=(20,15))
plt.show()
#






#Splitting the data into training and testing
train_set, test_set = split_train_test(housing, 0.2)
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print("Amount to train and test with")
print(len(train_set), "train +", len(test_set), ": test")
#







#Splitting data into training and test again based on index to avoid generating a different sets each run
housing_with_id = housing.reset_index() # adds an `index` column
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")
#







##Splitting data into training and test based on stratified sampling since our histograms show the incomes aggregate between 
##two and five and we dont want training data to be biased
#rounding up using ceil and merging all the categories greater than 5 into category 5
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)
#sample based on income category
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
 strat_train_set = housing.loc[train_index]
 strat_test_set = housing.loc[test_index]  
#print the proportions of income categories to see if sampling did the trick
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print("Income category proportions")
print(housing["income_cat"].value_counts() / len(housing) )
#remove the income_cat attribute so the data is in its original state
for set in (strat_train_set, strat_test_set):
 set.drop(["income_cat"], axis=1, inplace=True)
##
##







##Visualising the data further to gain insights
#create a copy of the data to play with without harming the training set
housing = strat_train_set.copy()
#creating a scatterplot of [population?] density over latitute and longitude
housing.plot(kind="scatter", x="longitude", y="latitude")
#setting the alpha option to 0.1 to more easily visualize high density areas
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)
#visualising housing prices
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
 s=housing["population"]/100, label="population",
 c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
)
plt.legend()
##




##Correlations
#creating a correlation matrix
corr_matrix = housing.corr()
#calculating correlation of each attribute with median house value
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print("correlation matrix")
print( corr_matrix["median_house_value"].sort_values(ascending=False) )
#plotting the correlation visualisations of a bunch of features
attributes = ["median_house_value", "median_income", "total_rooms",
 "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))
#zooming in on median house value since its visualisation appeared most appealing
housing.plot(kind="scatter", x="median_income", y="median_house_value",
 alpha=0.1)
#creating some new attributes
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]
#creating another correlation matrix
corr_matrix = housing.corr()
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print("new correlation matrix")
print( corr_matrix["median_house_value"].sort_values(ascending=False) ) 
#reverting to a clean training set by copying strat_train_set once again)- drop() creates a copy of the data 
#and does not affect strat_train_set
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()
##








#Fixing missing attribute in total_bedroom attribute bu setting values to mediam
median = housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median) 
imputer = Imputer(strategy="median")
housing_num = housing.drop("ocean_proximity", axis=1)
imputer.fit(housing_num)
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print("Imputer Statistics")
print(imputer.statistics_)
housing_num.median().values
X = imputer.transform(housing_num)
housing_tr = pd.DataFrame(X, columns=housing_num.columns)
#









#Handling Text using one-hot encoder
encoder = LabelEncoder()
housing_cat = housing["ocean_proximity"]
housing_cat_encoded = encoder.fit_transform(housing_cat)
housing_cat_encoded 
encoder = OneHotEncoder()
housing_cat_1hot = encoder.fit_transform(housing_cat_encoded.reshape(-1,1))
housing_cat_1hot
housing_cat_1hot.toarray()
encoder = LabelBinarizer()
housing_cat_1hot = encoder.fit_transform(housing_cat)
housing_cat_1hot
#







#Creation of Custom Transformers
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)
#    








#Pipeline Process
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]
num_pipeline = Pipeline([
 ('selector', DataFrameSelector(num_attribs)),
 ('imputer', Imputer(strategy="median")),
 ('attribs_adder', CombinedAttributesAdder()),
 ('std_scaler', StandardScaler()),
 ])
cat_pipeline = Pipeline([
 ('selector', DataFrameSelector(cat_attribs)),
 ('label_binarizer', MyLabelBinarizer()),
 ])
full_pipeline = FeatureUnion(transformer_list=[
 ("num_pipeline", num_pipeline),
 ("cat_pipeline", cat_pipeline),
 ])

housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared
housing_prepared.shape
##







##Training a linear regression model
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)
#Test on some instances from training set
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print("Linear Regressor Model [No cross val]")
print("Predictions:\t", lin_reg.predict(some_data_prepared))
print("Labels:\t\t", list(some_labels))
#Measure the regression using RMSE
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print("Root mean squared error = ", lin_rmse)
print(" ")
##







##Training a decision tree regressor
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print("Tree Regressor Model [No cross val] #will give zero error due to overfitting")
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)
#Evaluate on training set
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
print("Root mean squared error = ", tree_rmse)
print(" ")
##








##Training a tree regressor using cross validation feature
scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)
#Display the score
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print("Tree regressor scores [Cross Val based]")
display_scores(tree_rmse_scores)
##




##Training linear regressor with cross validation feature
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print("Linear regressor scores [Cross Val based]")
display_scores(lin_rmse_scores)
#



##Training random forest regressor with cross validation feature
forest_reg = RandomForestRegressor()
forest_scores = cross_val_score( forest_reg, housing_prepared, housing_labels,
                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print("Forest regressor scores [Cross Val based]")
display_scores(forest_rmse_scores)
##


# #Save the lin_reg, tree_reg and forest_reg models by using sklearn.externals.joblib
# from sklearn.externals import joblib
# joblib.dump(tree_reg, "tree_reg.pkl")
# joblib.dump(lin_reg, "lin_reg.pkl")
# joblib.dump(forest_reg, "forest_reg.pkl")
# #my_model_loaded = joblib.load("my_model.pkl")
# #



#Training using grid search to automate multiple cross validations 
#and pick the hyperperameters that give the best validation
#Training RandomForestRegressor with GridSearchCV
param_grid = [
 {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
 {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
 ]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
 scoring='neg_mean_squared_error')
print(grid_search)
grid_search.fit(housing_prepared, housing_labels)
#Save the best forest_reg model using joblib
joblib.dump(forest_reg, "forest_reg.pkl")
#Listing best parameters
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print("Best Parameter")
print(grid_search.best_params_)
#Listing best estimator 
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print("Best Estimator")
print(grid_search.best_estimator_)
#Printing all parameter values
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print("All paramter values and evaluation scores")
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
 print(np.sqrt(-mean_score), params)
#Indicating relative importance of each attribute to making accurate predictions, and importance scores
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_one_hot_attribs = list(encoder.classes_)
attributes = num_attribs + extra_attribs + cat_one_hot_attribs

print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print("Attribute importances for making accurate predictions")

print( sorted(zip(feature_importances, attributes), reverse=True) )
#Evaluating the system on test set
final_model = grid_search.best_estimator_
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()
X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print("Forest-reg Grid SearchCV rmse score")
print("Score =", final_rmse)
##




###Exercises

##Training SVM with kernel="linear" or kernel="rbf" with various values for the C and gamma
##hyperparameters 
param_grid_two = [
 {'kernel': ('linear', 'rbf' ), 'C': [3, 10, 30], 'gamma': [2, 4, 6, 8]} ]
print("...")
svm_reg = svm.SVC()
print("...")
grid_search_two = GridSearchCV(svm_reg, param_grid_two, scoring='neg_mean_squared_error')
print("...")
print(grid_search_two)
grid_search_two.fit(housing_prepared, housing_labels)
print(grid_search_two)
print("...")
#Save the best forest_reg model using joblib
print("...")
joblib.dump(svm_reg, "svm_reg.pkl")
print("...")
#Evaluating the system on test set
print("...")
final_model_two = grid_search_two.best_estimator_
print("...")
X_test_two = strat_test_set.drop("median_house_value", axis=1)
print("...")
y_test_two = strat_test_set["median_house_value"].copy()
print("...")
X_test_prepared_two = full_pipeline.transform(X_test_two)
print("...")
final_predictions_two = final_model.predict(X_test_prepared_two)
print("...")
svm_mse = mean_squared_error(y_test_two, final_predictions_two)
print("...")
svm_rmse = np.sqrt(svm_mse)
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print("SVM-based Grid SearchCV rmse score")
print("Score =", svm_rmse)
##


###
